**Preliminaries**

In [2]:
pip install numpy==1.23.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 23.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chex 0.1.86 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
pandas-stubs 2.0.3.230814 requires numpy>=1.25.0; python_version >= "3.9", but you have numpy 1.23.5 which is incompatible.


In [1]:
!git clone https://github.com/ShreyAgarwal11/Privacy-Preserving-Representation-for-Audio-Visual-Speech-Understanding.git
%cd /content/
!git clone https://github.com/facebookresearch/av_hubert.git

%cd av_hubert
!git submodule init
!git submodule update
!pip install scipy
!pip install sentencepiece
!pip install python_speech_features
!pip install scikit-video

%cd fairseq
!pip install ./

Cloning into 'Privacy-Preserving-Representation-for-Audio-Visual-Speech-Understanding'...
remote: Enumerating objects: 103723, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 103723 (delta 10), reused 0 (delta 0), pack-reused 103705
Receiving objects: 100% (103723/103723), 2.95 GiB | 35.63 MiB/s, done.
Resolving deltas: 100% (23/23), done.
Updating files: 100% (103015/103015), done.
/content
Cloning into 'av_hubert'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 149 (delta 18), reused 22 (delta 14), pack-reused 111
Receiving objects: 100% (149/149), 4.65 MiB | 10.39 MiB/s, done.
Resolving deltas: 100% (64/64), done.
/content/av_hubert
Submodule 'fairseq' (https://github.com/pytorch/fairseq) registered for path 'fairseq'
Cloning into '/content/av_hubert/fairseq'...
Submodule path 'fairseq': checked out 'afc77bdf4bb514

In [2]:
!mkdir -p /content/data/misc/
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2 -O /content/data/misc/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d /content/data/misc/shape_predictor_68_face_landmarks.dat.bz2
!wget --content-disposition https://github.com/mpc001/Lipreading_using_Temporal_Convolutional_Networks/raw/master/preprocessing/20words_mean_face.npy -O /content/data/misc/20words_mean_face.npy

--2024-04-23 20:29:28--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘/content/data/misc/shape_predictor_68_face_landmarks.dat.bz2’

/content/data/misc/ 100%[===================>]  61.07M  18.8MB/s    in 3.7s    

2024-04-23 20:29:32 (16.4 MB/s) - ‘/content/data/misc/shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]

--2024-04-23 20:29:44--  https://github.com/mpc001/Lipreading_using_Temporal_Convolutional_Networks/raw/master/preprocessing/20words_mean_face.npy
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mpc001/Lipreading_using_Temporal_Convolutional_Networks/master/preprocessing/20words_me

**Import a pre-trained model**

Fine tuned model -> Noise-Augmented AV-HuBERT Base

In [3]:
!pwd
%mkdir -p /content/data/
!wget https://dl.fbaipublicfiles.com/avhubert/model/lrs3_vox/avsr/base_noise_pt_noise_ft_433h.pt -O /content/data/finetune-model.pt

/content/av_hubert/fairseq
--2024-04-23 20:29:45--  https://dl.fbaipublicfiles.com/avhubert/model/lrs3_vox/avsr/base_noise_pt_noise_ft_433h.pt
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.226.210.78, 13.226.210.25, 13.226.210.15, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.226.210.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1928060481 (1.8G) [binary/octet-stream]
Saving to: ‘/content/data/finetune-model.pt’

/content/data/finet 100%[===================>]   1.79G   127MB/s    in 16s     

2024-04-23 20:30:01 (113 MB/s) - ‘/content/data/finetune-model.pt’ saved [1928060481/1928060481]



**Create Video Out of Frames**

In [4]:
import cv2
import os
import numpy as np

frame_folder = '/content/Privacy-Preserving-Representation-for-Audio-Visual-Speech-Understanding/VidTIMIT/fadg0/video/sa1'

output_video_path = '/content/output_video.mp4'

frame_rate = 25

frame_files = [f for f in os.listdir(frame_folder) if os.path.isfile(os.path.join(frame_folder, f))]

frame_files.sort()

video_resolution = (512, 384)

if video_resolution is None:
    first_frame_path = os.path.join(frame_folder, frame_files[0])
    first_frame = cv2.imread(first_frame_path)
    video_resolution = (first_frame.shape[1], first_frame.shape[0])


fourcc = cv2.VideoWriter_fourcc(*'MP4V')

out = cv2.VideoWriter(output_video_path, fourcc, frame_rate, video_resolution)

for frame_file in frame_files:
    frame_path = os.path.join(frame_folder, frame_file)
    frame = cv2.imread(frame_path)
    if (frame.shape[1], frame.shape[0]) != video_resolution:
        frame = cv2.resize(frame, video_resolution)
    out.write(frame)

out.release()


In [5]:
%cd /content/av_hubert/avhubert
import cv2
import tempfile
import torch
import utils as avhubert_utils
from argparse import Namespace
import fairseq
from fairseq import checkpoint_utils, options, tasks, utils
from IPython.display import HTML
from python_speech_features import logfbank
from scipy.io import wavfile

/content/av_hubert/avhubert


**Feature Extraction using AV-HUBERT**

In [6]:
def stacker(feats, stack_order):
            """
            Concatenating consecutive audio frames
            Args:
            feats - numpy.ndarray of shape [T, F]
            stack_order - int (number of neighboring frames to concatenate
            Returns:
            feats - numpy.ndarray of shape [T', F']
            """
            feat_dim = feats.shape[1]
            if len(feats) % stack_order != 0:
                res = stack_order - len(feats) % stack_order
                res = np.zeros([res, feat_dim]).astype(feats.dtype)
                feats = np.concatenate([feats, res], axis=0)
            feats = feats.reshape((-1, stack_order, feat_dim)).reshape(-1, stack_order*feat_dim)
            return feats

In [7]:
def extract_visual_feature(video_path, audio_path, ckpt_path, user_dir, is_finetune_ckpt=False):
  utils.import_user_module(Namespace(user_dir=user_dir))
  models, saved_cfg, task = checkpoint_utils.load_model_ensemble_and_task([ckpt_path])
  transform = avhubert_utils.Compose([
      avhubert_utils.Normalize(0.0, 255.0),
      avhubert_utils.CenterCrop((task.cfg.image_crop_size, task.cfg.image_crop_size)),
      avhubert_utils.Normalize(task.cfg.image_mean, task.cfg.image_std)])
  frames = avhubert_utils.load_video(video_path)
  print(f"Load video {video_path}: shape {frames.shape}")
  sample_rate, wav_data = wavfile.read(audio_path)
  audio_features = logfbank(wav_data, sample_rate).astype(np.float32)
  audio_features = stacker(audio_features, 4)
  print(f"Load audio {audio_path}: shape {audio_features.shape}")
  audio_features = torch.FloatTensor(audio_features).unsqueeze(dim=0).permute(0, 2, 1).cuda()
  frames = torch.FloatTensor(frames).unsqueeze(dim=0).unsqueeze(dim=0).cuda()
  if audio_features.shape[2] < frames.shape[2]:
    # Pad features_audio
    padding_size = frames.shape[2] - audio_features.shape[2]
    padding = torch.zeros((audio_features.shape[0], audio_features.shape[1], padding_size)).cuda()
    audio_features = torch.cat([audio_features, padding], dim=2)
  print(f"Load video {video_path}: shape {frames.shape}")
  print(f"Load audio {audio_path}: shape {audio_features.shape}")
  model = models[0]
  if hasattr(models[0], 'decoder'):
    print(f"Checkpoint: fine-tuned")
    model = models[0].encoder.w2v_model
  else:
    print(f"Checkpoint: pre-trained w/o fine-tuning")
  model.cuda()
  model.eval()
  with torch.no_grad():
    # Specify output_layer if you want to extract feature of an intermediate layer
    layer_features = []
    for i in range(12):
      feature, _ = model.extract_finetune(source={'video': frames, 'audio': audio_features}, padding_mask=None, output_layer=(i+1))
      layer_features.append(feature)
    feature, _ = model.extract_finetune(source={'video': frames, 'audio': audio_features}, padding_mask=None, output_layer=None)
    feature = feature.squeeze(dim=0)
  print(f"AvHuBert Feature shape: {feature.shape}")
  return layer_features, feature

mouth_roi_path, ckpt_path = "/content/output_video.mp4", "/content/data/finetune-model.pt"
audio_path = "/content/Privacy-Preserving-Representation-for-Audio-Visual-Speech-Understanding/VidTIMIT/fadg0/audio/sa1.wav"
user_dir = "/content/av_hubert/avhubert"
layer_features, feature = extract_visual_feature(mouth_roi_path, audio_path, ckpt_path, user_dir)

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Load video /content/output_video.mp4: shape (119, 384, 512)
Load audio /content/Privacy-Preserving-Representation-for-Audio-Visual-Speech-Understanding/VidTIMIT/fadg0/audio/sa1.wav: shape (119, 104)
Load video /content/output_video.mp4: shape torch.Size([1, 1, 119, 384, 512])
Load audio /content/Privacy-Preserving-Representation-for-Audio-Visual-Speech-Understanding/VidTIMIT/fadg0/audio/sa1.wav: shape torch.Size([1, 104, 119])
Checkpoint: fine-tuned
AvHuBert Feature shape: torch.Size([119, 768])


#Emotion Recognition

In [8]:
!unzip /content/drive/MyDrive/session1.zip
!unzip /content/drive/MyDrive/session1_video.zip

Archive:  /content/drive/MyDrive/session1.zip
   creating: session1/
   creating: session1/S01A/
   creating: session1/S01A/P/
  inflating: session1/S01A/P/MSP-IMPROV-S01A-F01-P-FM01.wav  
  inflating: session1/S01A/P/MSP-IMPROV-S01A-F01-P-FM02.wav  
  inflating: session1/S01A/P/MSP-IMPROV-S01A-F01-P-MF01.wav  
  inflating: session1/S01A/P/MSP-IMPROV-S01A-M01-P-FM01.wav  
  inflating: session1/S01A/P/MSP-IMPROV-S01A-M01-P-FM02.wav  
  inflating: session1/S01A/P/MSP-IMPROV-S01A-M01-P-MF01.wav  
  inflating: session1/S01A/P/MSP-IMPROV-S01A-M01-P-MF02.wav  
  inflating: session1/S01A/P/MSP-IMPROV-S01A-M01-P-MF03.wav  
   creating: session1/S01A/R/
  inflating: session1/S01A/R/MSP-IMPROV-S01A-M01-R-MM01.wav  
   creating: session1/S01A/S/
  inflating: session1/S01A/S/MSP-IMPROV-S01A-F01-S-FM01.wav  
  inflating: session1/S01A/S/MSP-IMPROV-S01A-F01-S-FM02.wav  
  inflating: session1/S01A/S/MSP-IMPROV-S01A-F01-S-FM03.wav  
  inflating: session1/S01A/S/MSP-IMPROV-S01A-F01-S-FM04.wav  
  infla

In [9]:
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
torch.cuda.empty_cache()

In [83]:
final_features = []

In [107]:
video_path = "session1/S01H/P/MSP-IMPROV-S01H-F01-P-MF01.avi"
audio_path = "session1/S01H/P/MSP-IMPROV-S01H-F01-P-MF01.wav"

In [108]:
import cv2

def resize_video(video_path, output_path, scale=0.5):
    # Open the video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file")
        return

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH) * scale)
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT) * scale)
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # You can change this to match your video codec

    # Create a VideoWriter object to write the video with new dimensions
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        resized_frame = cv2.resize(frame, (width, height))
        out.write(resized_frame)

    cap.release()
    out.release()

# Call the resize function
resized_video_path = "resized_video.avi"
resize_video(video_path, resized_video_path)

In [106]:
layer_features, feature = extract_visual_feature(resized_video_path, audio_path, ckpt_path, user_dir)

Load video resized_video.avi: shape (594, 540, 720)
Load audio session1/S01A/P/MSP-IMPROV-S01A-F01-P-MF01.wav: shape (496, 104)
Load video resized_video.avi: shape torch.Size([1, 1, 594, 540, 720])
Load audio session1/S01A/P/MSP-IMPROV-S01A-F01-P-MF01.wav: shape torch.Size([1, 104, 594])
Checkpoint: fine-tuned


OutOfMemoryError: CUDA out of memory. Tried to allocate 13.77 GiB. GPU 0 has a total capacity of 14.75 GiB of which 11.25 GiB is free. Process 10409 has 3.49 GiB memory in use. Of the allocated memory 1.27 GiB is allocated by PyTorch, and 2.07 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
final_features.append(feature)

In [109]:
labels = [0, 1, 0, 1]

In [34]:
import torch
from torch.utils.data import Dataset, DataLoader

class EmotionAVDataset(Dataset):
    def __init__(self, features, labels):
        """
        Args:
            features (list of Tensors): List where each element is a tensor of shape [sequence_length, feature_dim]
            labels (list of int): List of labels corresponding to each feature set
        """
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [110]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    # Separate features and labels
    features, labels = zip(*batch)
    # Pad features
    features_padded = pad_sequence(features, batch_first=True, padding_value=0)
    labels = torch.tensor(labels)
    return features_padded, labels

# Example usage
dataset = EmotionAVDataset(final_features, labels)
loader = DataLoader(dataset, batch_size=10, collate_fn=collate_fn, shuffle=True)

In [60]:
import torch.nn as nn

class EmotionClassifier(nn.Module):
    def __init__(self, feature_dim, hidden_dim, num_classes):
        super().__init__()
        self.lstm = nn.LSTM(feature_dim, hidden_dim, batch_first=True)
        self.classifier = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        out = self.classifier(hidden[-1])
        return out

In [119]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple LSTM-based model for emotion classification
class EmotionClassifier(nn.Module):
    def __init__(self, feature_dim, hidden_dim, num_classes):
        super(EmotionClassifier, self).__init__()
        self.lstm = nn.LSTM(feature_dim, hidden_dim, batch_first=True)
        self.classifier = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        out = self.classifier(hidden[-1])  # Use the last hidden state
        return out

# Assume num_features and num_classes are defined based on your dataset
feature_dim = 768  # Feature dimension for each timestep
hidden_dim = 128  # Hidden dimension for LSTM
num_classes = 2   # Example: 5 different emotion classes

model = EmotionClassifier(feature_dim, hidden_dim, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)

In [120]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [121]:
model = EmotionClassifier(feature_dim, hidden_dim, num_classes).to(device)

In [122]:
import torch

def adjust_learning_rate(optimizer, epoch, initial_lr=0.001, lr_decay_epoch=3):
    """Sets the learning rate to the initial LR decayed by 10 every lr_decay_epoch epochs"""
    lr = initial_lr * (0.1**(epoch // lr_decay_epoch))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def train_model(num_epochs, initial_lr=0.001, lr_decay_epoch=3):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=initial_lr)
    for epoch in range(num_epochs):
        running_loss = 0.0
        adjust_learning_rate(optimizer, epoch, initial_lr, lr_decay_epoch)
        for features, labels in loader:
            features, labels = features.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Optional: Print average gradients per epoch to check for vanishing/exploding gradients
        avg_gradients = {name: torch.mean(param.grad.abs()).item() for name, param in model.named_parameters() if param.grad is not None}
        print(f"Epoch {epoch+1}, Loss: {running_loss / len(loader)}, Avg Gradients: {avg_gradients}")

        # Check for NaN loss
        if torch.isnan(loss).any():
            print("NaN loss detected")
            break

def evaluate_model():
    model.eval()
    total_correct = 0
    total_samples = 0
    with torch.no_grad():
        for features, labels in loader:
            features, labels = features.to(device), labels.to(device)
            outputs = model(features)
            _, predicted = torch.max(outputs, 1)
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)
    accuracy = total_correct / total_samples
    print(f"Accuracy: {accuracy:.2f}")


In [123]:
train_model(50)
evaluate_model()

Epoch 1, Loss: 0.7050982117652893, Avg Gradients: {'lstm.weight_ih_l0': 0.00015319950762204826, 'lstm.weight_hh_l0': 0.0001138313818955794, 'lstm.bias_ih_l0': 0.00045707906247116625, 'lstm.bias_hh_l0': 0.00045707906247116625, 'classifier.weight': 0.010660267435014248, 'classifier.bias': 0.004405215382575989}
Epoch 2, Loss: 0.6446354389190674, Avg Gradients: {'lstm.weight_ih_l0': 0.00011407291458453983, 'lstm.weight_hh_l0': 9.138100722339004e-05, 'lstm.bias_ih_l0': 0.0005290161934681237, 'lstm.bias_hh_l0': 0.0005290161934681237, 'classifier.weight': 0.010138976387679577, 'classifier.bias': 0.028205230832099915}
Epoch 3, Loss: 0.6003038883209229, Avg Gradients: {'lstm.weight_ih_l0': 8.434417395619676e-05, 'lstm.weight_hh_l0': 8.907501614885405e-05, 'lstm.bias_ih_l0': 0.0007479123887605965, 'lstm.bias_hh_l0': 0.0007479123887605965, 'classifier.weight': 0.010165289044380188, 'classifier.bias': 0.055923957377672195}
Epoch 4, Loss: 0.5680534839630127, Avg Gradients: {'lstm.weight_ih_l0': 5.7